In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Khadija\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk import word_tokenize, sent_tokenize
from nltk import FreqDist

In [ ]:
import sqlite3
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

# Download NLTK WordNet data if not already downloaded
nltk.download('wordnet')

# SQLite database file
db_file = 'products.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * FROM products", conn)

# Function to get synonyms using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

In [ ]:
text = "give me smartphones with price less than 50000"


In [ ]:
numeric_columns = ['Price', 'Rating']  # Add other relevant columns
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')


In [ ]:
def price_filter(ran, flag):
    if len(ran) > 1 and type(ran) == list:
        return df[df['Price'] < ran[0] and df['Price'] > ran[1]]
    elif flag == True:
        return df[df['Price'] < ran]
    elif flag == False:
        return df[df['Price'] > ran]

def rating_filter(ran, flag):
    if len(ran) > 1 and type(ran) == list:
        return df[df['Rating'] < ran[0] and df['Rating'] > ran[1]]
    elif flag == True:
        return df[df['Rating'] < ran]
    elif flag == False:
        return df[df['Rating'] > ran]

In [ ]:
def filter_data(query):
    # Tokenize the user query
    tokens = word_tokenize(query.lower()) 

    # # Initialize filters
    # price_filter = df['Price'] >= 0
    # ratings_filter = df['Rating'] >= 0
    # brand_filter = df['Brand'].notnull()
    # product_name_filter = df['Name'].notnull()

    # Flags for handling specific cases
    price_range_flag = False
    specs_flag = False
    specific_phone_flag = False

    # Check for keywords and handle specific cases
    for i, token in enumerate(tokens):
        if token in get_synonyms('below'):
            num = []
            current_number = ''
            for t in token:
                if t.isdigit():
                    current_number += t
                elif current_number:
                    num.append(current_number)
                    current_number = ''

            # Check if there is a number after processing the current token
            if current_number:
                num.append(current_number)

            # Your code here with the 'num' list
            print(f"Token '{token}' does not contain 'and', and the list of numbers is: {num}")

            if token in get_synonyms('price') and token not in get_synonyms('rating'):
                if len(num) < 2:
                    new_df = price_filter(num[0], True)
                else:
                    new_df = price_filter(num)
            elif token in get_synonyms('rating') and token not in get_synonyms('price'):
                if len(num) < 2:
                    new_df = rating_filter(num[0], True)
                else:
                    new_df = rating_filter(num)
            
                

        elif token in get_synonyms('above'):
            if i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
                threshold = float(tokens[i + 1])
                price_filter &= (df['Price'] > threshold)

        elif token in get_synonyms('between'):
            if i + 3 < len(tokens) and tokens[i + 1].isdigit() and \
                    tokens[i + 2] in get_synonyms('and') and \
                    tokens[i + 3].isdigit():
                lower_threshold = float(tokens[i + 1])
                upper_threshold = float(tokens[i + 3])
                price_filter &= (df['Price'] >= lower_threshold) & (df['Price'] <= upper_threshold)
                price_range_flag = True

        elif token.isdigit() and i + 1 < len(tokens) and tokens[i + 1] in get_synonyms('and'):
            if i + 2 < len(tokens) and tokens[i + 2].replace('.', '', 1).isdigit():
                lower_threshold = float(token)
                upper_threshold = float(tokens[i + 2])
                price_filter &= (df['Price'] >= lower_threshold) & (df['Price'] <= upper_threshold)
                price_range_flag = True

        elif token == 'under' and i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
            threshold = float(tokens[i + 1])
            price_filter &= (df['Price'] < threshold)
            price_range_flag = True

        elif token == 'over' and i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
            threshold = float(tokens[i + 1])
            price_filter &= (df['Price'] > threshold)
            price_range_flag = True

        elif token == 'best' and i + 4 < len(tokens) and tokens[i + 1] == 'phones' and tokens[i + 2] == 'with' and \
                tokens[i + 3] == 'respect' and tokens[i + 4] == 'to':
            specs_flag = True

        elif specs_flag and token.isdigit() and i + 2 < len(tokens) and tokens[i + 1] == 'gb' and tokens[i + 2] == 'ram':
            gb_ram = int(token)
            product_name_filter &= (df['RAM'] == gb_ram)

        elif specs_flag and token.isdigit() and i + 2 < len(tokens) and tokens[i + 1] == 'mp' and tokens[i + 2] == 'camera':
            mp_camera = int(token)
            product_name_filter &= (df['Camera'] == mp_camera)

        elif specs_flag and token == 'pta' and i + 2 < len(tokens) and tokens[i + 1] == 'approved':
            pta_approved = tokens[i + 2]
            product_name_filter &= (df['PTA Approved'].str.lower() == pta_approved)

        elif token == 'phone' and i + 1 < len(tokens):
            specific_phone = tokens[i + 1]
            product_name_filter &= (df['Name'].str.lower().str.contains(specific_phone))
            specific_phone_flag = True

        elif token == 'product' and i + 1 < len(tokens) and tokens[i + 1] == 'name':
            specific_product_name = tokens[i + 2]
            product_name_filter &= (df['Name'].str.lower().str.contains(specific_product_name))
            specific_phone_flag = True

    # Apply filters
    filtered_data = df[price_filter & ratings_filter & brand_filter & product_name_filter]

    if price_range_flag:
        print("Bot: Here are the results within the specified price range:")
    elif specs_flag:
        print("Bot: Here are the best phones with the specified specs:")
    elif specific_phone_flag:
        print(f"Bot: Here are the results for {specific_phone}:")
    elif not filtered_data.empty:
        print("Bot: Here are the matching results:")
        print(filtered_data)
    else:
        print("Bot: Sorry, I couldn't understand the query or no results matched.")


In [2]:


# Function to filter data based on user queries
def filter_data(query):
    # Tokenize the user query
    tokens = word_tokenize(query.lower()) 

    # Initialize filters
    price_filter = df['Price'] >= 0
    ratings_filter = df['Rating'] >= 0
    brand_filter = df['Brand'].notnull()
    product_name_filter = df['Name'].notnull()

    # Flags for handling specific cases
    price_range_flag = False
    specs_flag = False
    specific_phone_flag = False

    # Check for keywords and handle specific cases
    for i, token in enumerate(tokens):
        if token in get_synonyms('below'):
            if i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
                threshold = float(tokens[i + 1])
                price_filter &= (df['Price'] < threshold)

        elif token in get_synonyms('above'):
            if i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
                threshold = float(tokens[i + 1])
                price_filter &= (df['Price'] > threshold)

        elif token in get_synonyms('between'):
            if i + 3 < len(tokens) and tokens[i + 1].isdigit() and \
                    tokens[i + 2] in get_synonyms('and') and \
                    tokens[i + 3].isdigit():
                lower_threshold = float(tokens[i + 1])
                upper_threshold = float(tokens[i + 3])
                price_filter &= (df['Price'] >= lower_threshold) & (df['Price'] <= upper_threshold)
                price_range_flag = True

        elif token.isdigit() and i + 1 < len(tokens) and tokens[i + 1] in get_synonyms('and'):
            if i + 2 < len(tokens) and tokens[i + 2].replace('.', '', 1).isdigit():
                lower_threshold = float(token)
                upper_threshold = float(tokens[i + 2])
                price_filter &= (df['Price'] >= lower_threshold) & (df['Price'] <= upper_threshold)
                price_range_flag = True

        elif token == 'under' and i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
            threshold = float(tokens[i + 1])
            price_filter &= (df['Price'] < threshold)
            price_range_flag = True

        elif token == 'over' and i + 1 < len(tokens) and tokens[i + 1].replace('.', '', 1).isdigit():
            threshold = float(tokens[i + 1])
            price_filter &= (df['Price'] > threshold)
            price_range_flag = True

        elif token == 'best' and i + 4 < len(tokens) and tokens[i + 1] == 'phones' and tokens[i + 2] == 'with' and \
                tokens[i + 3] == 'respect' and tokens[i + 4] == 'to':
            specs_flag = True

        elif specs_flag and token.isdigit() and i + 2 < len(tokens) and tokens[i + 1] == 'gb' and tokens[i + 2] == 'ram':
            gb_ram = int(token)
            product_name_filter &= (df['RAM'] == gb_ram)

        elif specs_flag and token.isdigit() and i + 2 < len(tokens) and tokens[i + 1] == 'mp' and tokens[i + 2] == 'camera':
            mp_camera = int(token)
            product_name_filter &= (df['Camera'] == mp_camera)

        elif specs_flag and token == 'pta' and i + 2 < len(tokens) and tokens[i + 1] == 'approved':
            pta_approved = tokens[i + 2]
            product_name_filter &= (df['PTA Approved'].str.lower() == pta_approved)

        elif token == 'phone' and i + 1 < len(tokens):
            specific_phone = tokens[i + 1]
            product_name_filter &= (df['Name'].str.lower().str.contains(specific_phone))
            specific_phone_flag = True

        elif token == 'product' and i + 1 < len(tokens) and tokens[i + 1] == 'name':
            specific_product_name = tokens[i + 2]
            product_name_filter &= (df['Name'].str.lower().str.contains(specific_product_name))
            specific_phone_flag = True

    # Apply filters
    filtered_data = df[price_filter & ratings_filter & brand_filter & product_name_filter]

    if price_range_flag:
        print("Bot: Here are the results within the specified price range:")
    elif specs_flag:
        print("Bot: Here are the best phones with the specified specs:")
    elif specific_phone_flag:
        print(f"Bot: Here are the results for {specific_phone}:")
    elif not filtered_data.empty:
        print("Bot: Here are the matching results:")
        print(filtered_data)
    else:
        print("Bot: Sorry, I couldn't understand the query or no results matched.")

# Chatbot loop
while True:
    user_input = input("User: ")
    
    if user_input.lower() == 'exit':
        print("Exiting the chatbot.")
        break

    # Filter data based on user input
    results = filter_data(user_input)

    # Display results
    # if not results.empty:
    print("Bot: Here are the matching results:")
    print(results)
    # else:
    #     print("Bot: Sorry, I couldn't understand the query or no results matched.")

# Close the connection
conn.close()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Khadija\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Bot: Here are the results within the specified price range:
Bot: Here are the matching results:
None
Exiting the chatbot.
